In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import pandas as pd
import numpy as np
from datetime import datetime
import time
from kapteyn import wcs

import warnings
warnings.filterwarnings("ignore")

In [2]:
## This function allows to execute the OS commands
def xcmd(cmd, verbose=True):
    """Runs an OS command
    :param cmd: terminal command
    :type cmd: ``str``
    :param verbose: printing the details, default True 
    :type verbose: ``boolean``
    :return: OS outputs
    :rtype: ``str``
    """

    if verbose: print('\n'+cmd)

    tmp=os.popen(cmd)
    output=''
    for x in tmp: output+=x
    if 'abort' in output:
        failure=True
    else:
        failure=tmp.close()
    if False:
        print('execution of %s failed' % cmd)
        print('error is as follows', output)
        sys.exit()
    else:
        return output


In [3]:
# !scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files ../.

In [4]:
# !scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat* ../bar_files/.

In [5]:
# This function works with both dbtable and the catalog header name on EDD
def get_catal(catal_string, infoFile="../bar_files/catalogs_info.dat", bars_folder='../bar_files/'):
    
    xcmd("scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.", verbose=True)

    with open(infoFile, "r") as f:
        lines = f.readlines()


        try:
            for i, line in enumerate(lines):
                if catal_string in line:
                    
                    while not "begin" in line:
                        i-=1
                        line = lines[i]
                    while not "filename" in line:
                        i+=1
                        line=lines[i]

                    fname_string = line
                    break
        except: 
            return "", []
            
        catal_columns = []
        
        try:
            while not "column" in line:
                i+=1
                line=lines[i]
            while not 'end' in line:
                catal_columns.append(line.split("=>")[1].split("|")[0].strip())
                i+=1
                line=lines[i]
                
        except:
            print("why")
            return "", []


    catal_file_name = fname_string.split("=>")[1].strip("\n").strip("")
    
    xcmd("scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/"+catal_file_name+"  "+bars_folder+"/.", verbose=True)
    
    data = pd.read_csv(bars_folder+catal_file_name, names=catal_columns, delimiter='|')
    
    for col in catal_columns:
        if col.upper()=="PGC":
            break
    
    data.rename(columns={col:col.upper()}, inplace=True)
    data = data.set_index('PGC')
    
    # taking care of empty string
    data = data.replace(r'^\s*$', np.nan, regex=True)

    
    return data

In [6]:
data = {}
catalogs = []

In [7]:
catal_dbtable = "CF4 All Calibrator Groups"

data = get_catal(catal_dbtable)
data.head(20)


scp -r ehsan@arp.ifa.hawaii.edu:~/catalogs_info.dat ../bar_files/.

scp -r ehsan@arp.ifa.hawaii.edu:~/bar_files/3ZPcalib_groups_bar  ../bar_files//.


,N,DM,rms,StDv,Ng,Vcmb,glon,glat,sgl,sgb
PGC,,,,,,,,,,
35,1,29.730,NaN,NaN,1,28,112.3102,-22.3207,333.3464,20.3429
143,1,24.910,NaN,NaN,1,-455,75.8635,-73.6245,277.8077,8.0860
218,1,30.730,NaN,NaN,6,607,106.4391,-45.1797,309.0668,16.3813
388,1,29.650,NaN,NaN,1,431,24.8340,-79.7884,266.0981,3.1192
591,1,29.510,NaN,NaN,1,5,114.0867,-21.2440,334.9193,19.0438
621,1,27.480,NaN,NaN,1,-46,351.4759,-78.1170,260.1830,0.4032
701,1,29.270,NaN,NaN,1,241,43.6860,-80.4343,269.3872,3.2262
930,1,29.060,NaN,NaN,1,162,55.9027,-80.6717,271.3799,2.9060
1014,2,26.530,0.058,0.041,2,-127,332.8690,-75.7300,256.2474,-2.3622


In [8]:
cc = []
for c in data.columns[:6]:
    cc.append(c)
cc += ["ra", "dec"]
for c in data.columns[6:]:
    cc.append(c)

cc

['N',
 'DM',
 'rms',
 'StDv',
 'Ng',
 'Vcmb',
 'ra',
 'dec',
 'glon',
 'glat',
 'sgl',
 'sgb']

In [9]:
from kapteyn import wcs

super2celestial = wcs.Transformation("supergalactic j2000 j2000", "equatorial")
# RA, DEC = ((sgl, sgb))
super2galactic = wcs.Transformation("supergalactic j2000 j2000", "galactic")
# gl, gb = tran((sgl, sgb))

In [10]:
data["ra"] = data.apply(lambda x: super2celestial((x.sgl,x.sgb))[0], axis=1)
data["dec"] = data.apply(lambda x: super2celestial((x.sgl,x.sgb))[1], axis=1)

In [11]:
data = data[cc]
data.head()

,N,DM,rms,StDv,Ng,Vcmb,ra,dec,glon,glat,sgl,sgb
PGC,,,,,,,,,,,,
35,1,29.73,NaN,NaN,1,28,0.093779,39.495506,112.3102,-22.3207,333.3464,20.3429
143,1,24.91,NaN,NaN,1,-455,0.492585,-15.460969,75.8635,-73.6245,277.8077,8.0860
218,1,30.73,NaN,NaN,6,607,0.834949,16.146202,106.4391,-45.1797,309.0668,16.3813
388,1,29.65,NaN,NaN,1,431,1.382951,-28.098427,24.8340,-79.7884,266.0981,3.1192
591,1,29.51,NaN,NaN,1,5,1.933539,40.875915,114.0867,-21.2440,334.9193,19.0438


In [13]:
data[cc].to_csv("tmp.csv", sep='|')
table   = np.genfromtxt('tmp.csv' , delimiter='|', 
                        filling_values=-10000, names=True, dtype=None, encoding=None)

colnames = table.dtype.names

## table is a structured array
myTable = {}
for name in table.dtype.names:
    myTable[name] = table[name]
table = myTable
## table is now a dictionary

myTable = Table()

for key in colnames:
    if key in ["SNIa"]:
        myTable.add_column(Column(data=table[key], name=key))
    elif key in ["PGC", "PGC1", "IDT14g", 'Ng', 'N']:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int))) 
    elif key in ["Vcmb", "Vmod"]:
        myTable.add_column(Column(data=table[key], name=key, dtype=np.dtype(int)))
    elif key in ["sgl", "sgb", "ra", "dec", "gl", "gb"]:
        myTable.add_column(Column(data=table[key], name=key, format='%0.4f'))
    else:
        myTable.add_column(Column(data=table[key], name=key, format='%0.3f'))

## to be used on EDD
myTable.write('3ZPcalib_groups_bar_updated', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True)